In [8]:
import os
import glob

import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import io

from tensorflow import keras
import tensorflow as tf

# gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.6)
# sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [9]:
TRAIN_DIR = '../Dataset/training_set/training_set/'
TEST_DIR = '../Dataset/test_set/test_set/'

items = ['cats', 'dogs']
SIZE = (120, 120, )

X_train = []
X_test = []
Y_train = []
Y_test = []


In [10]:
'''
Reading Training Data
'''
cats = []
dogs = []

for e in items:
    _dir = os.path.join(TRAIN_DIR, e)

    for path in glob.glob(_dir + '/*.jpg'):
        img = io.imread(path, as_gray= True)
        img = cv2.resize(img, SIZE)
        if e == 'cats':
            cats.append(img)
        else:
            dogs.append(img)
        

In [14]:
cats = np.array(cats)
dogs = np.array(dogs)

In [15]:
cats = np.expand_dims(cats, axis= -1)
dogs = np.expand_dims(dogs, axis= -1)

In [16]:
'''
Data Augmentation
'''
from keras.preprocessing.image import ImageDataGenerator

gen = ImageDataGenerator(
    rotation_range= 45,
    height_shift_range= .2,
    width_shift_range= .2,
    zoom_range= .2,
    vertical_flip= True,
    horizontal_flip= True,
    fill_mode= 'reflect',
)
BATCH_LEN = int(1e4)
i = 0

for batch in gen.flow(cats):
    for img in batch:
        X_train.append(img)
    i += 1
    if i >= BATCH_LEN:
        break

KeyboardInterrupt: 

In [7]:
for _ in range(len(cats)):
    Y_train.append(0)

In [8]:
BATCH_LEN = int(1e4)
i = 0

for batch in gen.flow(dogs):
    for img in batch:
        X_train.append(img)
        Y_train.append(1)
    i += 1
    if i >= BATCH_LEN:
        break

In [28]:
X_train = np.expand_dims(X_train, axis= -1)
X_test = np.expand_dims(X_test, axis= -1)

In [23]:
'''
Reading testing data
'''
for e in items:
    _dir = os.path.join(TEST_DIR, e)

    for path in glob.glob(_dir + '/*.jpg'):
        img = io.imread(path, as_gray= True)
        img = cv2.resize(img, SIZE)
        X_test.append(img)
        Y_test.append(items.index(e))
        

X_test = np.array(X_test)
Y_test = np.array(Y_test)


In [5]:
from sklearn.utils import shuffle

X_train, Y_train = shuffle(X_train, Y_train)
X_test, Y_test = shuffle(X_test, Y_test)

In [30]:
X_train.shape , X_train.max()

((8005, 120, 120, 1), 1.0)

In [7]:
X_train = tf.expand_dims(X_train, -1)
X_test = tf.expand_dims(X_test, -1)
Y_train = keras.utils.to_categorical(Y_train, 2)
Y_test = keras.utils.to_categorical(Y_test, 2)

In [8]:
X_train[0]

<tf.Tensor: shape=(120, 120, 1), dtype=float64, numpy=
array([[[0.63909475],
        [0.66116059],
        [0.55402031],
        ...,
        [0.16786187],
        [0.25181365],
        [0.24159663]],

       [[0.56346215],
        [0.51141357],
        [0.42808163],
        ...,
        [0.19906013],
        [0.26393129],
        [0.26213911]],

       [[0.56058871],
        [0.52194941],
        [0.36995547],
        ...,
        [0.18677255],
        [0.26931692],
        [0.2577622 ]],

       ...,

       [[0.39558471],
        [0.30967464],
        [0.1474702 ],
        ...,
        [0.63844048],
        [0.64033046],
        [0.63967686]],

       [[0.14626444],
        [0.29730863],
        [0.11093425],
        ...,
        [0.63818993],
        [0.64353961],
        [0.64352   ]],

       [[0.40693329],
        [0.42934131],
        [0.08559656],
        ...,
        [0.65086967],
        [0.65536314],
        [0.65536314]]])>

In [9]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), strides= (1, 1), input_shape= X_train.shape[1:], activation= 'selu'),
    keras.layers.MaxPool2D(pool_size= (3, 3)),

    keras.layers.Conv2D(24, (3, 3), strides= (2, 2), activation= 'selu'),
    keras.layers.MaxPool2D(pool_size= (2, 2)),

    tf.keras.layers.GlobalMaxPool2D(),

    keras.layers.Dense(16, activation= 'relu'),
    keras.layers.Dense(8, activation= 'relu'),
    keras.layers.Dense(2, activation= 'softmax')

])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 118, 118, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 39, 39, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 19, 19, 24)        6936      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 9, 9, 24)         0         
 2D)                                                             
                                                                 
 global_max_pooling2d (Globa  (None, 24)               0         
 lMaxPooling2D)                                                  
                                                        

In [14]:
model.compile(optimizer= keras.optimizers.Adam(), loss= keras.losses.categorical_crossentropy, metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size= 32, epochs= 10, validation_data= (X_test, Y_test))

Epoch 1/10
251/251 [==============================] - 5s 17ms/step - loss: 0.5251 - accuracy: 0.7428 - val_loss: 0.6105 - val_accuracy: 0.6777
Epoch 2/10
251/251 [==============================] - 4s 15ms/step - loss: 0.5199 - accuracy: 0.7418 - val_loss: 0.6222 - val_accuracy: 0.6787
Epoch 3/10
251/251 [==============================] - 4s 15ms/step - loss: 0.5115 - accuracy: 0.7443 - val_loss: 0.6010 - val_accuracy: 0.6906
Epoch 4/10
251/251 [==============================] - 4s 15ms/step - loss: 0.5047 - accuracy: 0.7535 - val_loss: 0.5984 - val_accuracy: 0.6960
Epoch 5/10
251/251 [==============================] - 4s 15ms/step - loss: 0.4936 - accuracy: 0.7646 - val_loss: 0.6127 - val_accuracy: 0.6975
Epoch 6/10
251/251 [==============================] - 4s 15ms/step - loss: 0.4905 - accuracy: 0.7620 - val_loss: 0.6388 - val_accuracy: 0.6772
Epoch 7/10
251/251 [==============================] - 4s 15ms/step - loss: 0.4840 - accuracy: 0.7683 - val_loss: 0.6090 - val_accuracy: 0.6901